# Gray Wolf Optimization (GWO)

Passada a fase de exploração agora é possível fazer o uso do GWO para clusterização das flores no dataset apresentado.


>

## Inicializando o ambiente

A célula abaixo prepara o ambiente colab para utilizar o diretório do projeto no Google Drive.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/My Drive/Github/iris-clustering/

!pip install -q --upgrade plotly 
!pip install -q jupyter-dash

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Github/iris-clustering


## Carrengando módulos e pacotes. 

A célula abaixo carrega os pacotes necessários.

> **OBS:** De forma a tornar a apresentação mais enxuta toda a lógica foi implementada em um módulo separado aos notebooks. Chamada `main.py`

In [2]:
import pandas as pd
import numpy as np

from sklearn import datasets

import main
#help(main)

## Carregando o Dataset

A célula abaixo prepara a base de dados para ser utilizada na classificação.

In [3]:
iris = datasets.load_iris()

iris_data   = pd.DataFrame((iris.data),columns=['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width'])
iris_target = pd.DataFrame((iris.target), columns=['Species'])

iris = pd.concat([iris_data, iris_target], axis=1)
iris_dic = dict([('Setosa', 0),('Versicolor',1),('Virginica',2)])

## Separando os dados

De forma conveniente, vamos utilizar a classificação já realizada da plantas como referência. 

Para isso a célula abaixo separa os dados e os índices correspondentes.

In [4]:
setosa_data, setosa_idx = main.separaDados(iris_data, iris_target, iris_dic['Setosa'])
versicolor_data, versicolor_idx = main.separaDados(iris_data, iris_target, iris_dic['Versicolor'])
virginica_data, virginica_idx = main.separaDados(iris_data, iris_target, iris_dic['Virginica'])

## Obtendo a Covariância

Sendo a covariância uma medida do grau de interdependência (ou inter-relação) numérica entre duas variáveis aleatórias, essa medida estatística é uma boa candidata a ser utilizada como variável de otimização do problema.

Como já existem classes conhecidas a covariância desses grupos será utilizada como benchmark para a otimização.

In [5]:
cov_setosa = np.cov(np.array(setosa_data).T)
cov_versicolor = np.cov(np.array(versicolor_data).T)
cov_virginica = np.cov(np.array(virginica_data).T)


## Treinando o modelo para as flores

Vamos otimizar a média e a covariancia ao mesmo tempo, e treinar um modelo que seja capaz de classificar as plantas do dataset.

### Setando os parâmetros

In [6]:
# Individuo = [med1 med2 med3 med4 cov1 cov2 cov3 cov4]
lb_mean = [-10,-10,-10,-10]
ub_mean = [10,10,10,10]
#lb_cov  = [-1, -1, -1, -1]
#ub_cov  = [1, 1, 1, 1]

#lb = lb_mean + lb_cov
#ub = ub_mean + ub_cov

N_iteracao = 200
a = 0.1
N_lobos = 100

### Treinando Setosa

In [7]:
X_setosa,fval_setosa,V_fob_setosa,medFob_setosa = main.GWO(lb_mean,ub_mean,N_iteracao,a,N_lobos,cov_setosa,setosa_data)
print("Média Otimizada Obtida - Setosa: {}".format(X_setosa))

Média Otimizada Obtida - Setosa: [4.97300642 3.35546065 1.45680942 0.20993886]


### Treinando Versicolor

In [8]:
X_versicolor,fval_versicolor,V_fob_versicolor, medFob_versicolor = main.GWO(lb_mean,ub_mean,N_iteracao,a,N_lobos,cov_versicolor,versicolor_data)
print("Média Otimizada Obtida - Versicolor: {}".format(X_versicolor))

Média Otimizada Obtida - Versicolor: [5.94606546 2.81312926 4.22688333 1.31031525]


### Treinando Virginica

In [9]:
X_virginica,fval_virginica,V_fob_virginica, medFob_virginica = main.GWO(lb_mean,ub_mean,N_iteracao,a,N_lobos,cov_virginica, virginica_data)
print("Média Otimizada Obtida - Versicolor: {}".format(X_virginica))

Média Otimizada Obtida - Versicolor: [6.51128535 2.98182851 5.45258968 2.06247619]


### Gráfico do processo

Os gráficos abaixo indicam a evolução dos processos de otimização do modelo

In [11]:
main.plot_trainning(V_fob_setosa, V_fob_versicolor, V_fob_virginica)

## Executando a classificação

> Empiricamente o valor de 0.01 foi escolhido como o limiar entre uma boa probabilidade e uma ruim.

In [19]:
Pr_setosa = main.classify(X_setosa, cov_setosa, iris_data)
Pr_versicolor = main.classify(X_versicolor, cov_versicolor, iris_data)
Pr_virginica = main.classify(X_virginica, cov_virginica, iris_data)

# Conta quantas plantas tem a probabilidade maior que 0.01
C_setosa  = np.count_nonzero(np.array(Pr)>0.01)
C_versicolor  = np.count_nonzero(np.array(Pr)>0.01)
C_virginica  = np.count_nonzero(np.array(Pr)>0.01)


# Encontra a posição das plantas que foram classificadas com probabilidade > 0.01
value_setosa = np.where(np.array(Pr_setosa)>0.01)
value_versicolor = np.where(np.array(Pr_versicolor)>0.01)
value_virginica = np.where(np.array(Pr_virginica)>0.01)

Arr_setosa   = np.array(value_setosa)
Arr_versicolor   = np.array(value_versicolor)
Arr_virginica   = np.array(value_virginica)

## Visualizando os Resultados

In [52]:
x_setosa = np.array(iris_data)[Arr_setosa[:],0]
y_setosa = np.array(iris_data)[Arr_setosa[:],2]

x_versicolor = np.array(iris_data)[Arr_versicolor[:],0]
y_versicolor = np.array(iris_data)[Arr_versicolor[:],2]

x_virginica = np.array(iris_data)[Arr_virginica[:],0]
y_virginica = np.array(iris_data)[Arr_virginica[:],2]

In [66]:
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots

fig = make_subplots(rows=3, cols=1, subplot_titles=("Setosa", "Versicolor", "Virginica"))

fig.add_trace(go.Scatter(x=x_setosa[0],y=y_setosa[0],mode='markers', name='Classificado', marker_symbol="circle"), row=1, col=1)
fig.add_trace(go.Scatter(x=np.array(iris_data)[setosa_idx,0], y=np.array(iris_data)[setosa_idx,2], mode='markers', name='Target', marker_symbol="cross"), row=1, col=1)

fig.add_trace(go.Scatter(x=x_versicolor[0],y=y_versicolor[0],mode='markers', name='Classificado', marker_symbol="circle"), row=2, col=1)
fig.add_trace(go.Scatter(x=np.array(iris_data)[versicolor_idx,0], y=np.array(iris_data)[versicolor_idx,2], mode='markers', name='Target', marker_symbol="cross"), row=2, col=1)

fig.add_trace(go.Scatter(x=x_virginica[0],y=y_virginica[0],mode='markers', name='Classificado', marker_symbol="circle"), row=3, col=1)
fig.add_trace(go.Scatter(x=np.array(iris_data)[virginica_idx,0], y=np.array(iris_data)[virginica_idx,2], mode='markers', name='Target', marker_symbol="cross"), row=3, col=1)

# Update xaxis properties
fig.update_xaxes(title_text="Comprimento (cm)", row=3, col=1)

# Update yaxis properties
fig.update_yaxes(title_text="Largura (cm)", row=1, col=1)
fig.update_yaxes(title_text="Largura (cm)", row=2, col=1)
fig.update_yaxes(title_text="Largura (cm)", row=3, col=1)

fig.update_layout(height=800, title_text="Resultado da Classificação")
fig.show()